In [1]:
! pip install transformers

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 6.5 MB 67.1 MB/s 
     |████████████████████████████████| 77 kB 7.2 MB/s 
     |████████████████████████████████| 596 kB 75.9 MB/s 
     |████████████████████████████████| 895 kB 72.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O KorQuAD_v1.0_dev.json

--2022-04-07 05:26:15--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.110.153, 185.199.108.153, 185.199.109.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M   197MB/s    in 0.2s    

2022-04-07 05:26:15 (197 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2022-04-07 05:26:15--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.110.153, 185.199.108.153, 185.199.109.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[======

In [3]:
import os
import json
import numpy as np
from tqdm import tqdm
from pathlib import Path
from transformers import BertTokenizerFast
import tensorflow as tf

In [5]:
# KorQuAD
# def read_squad(path):
#     path = Path(path)
#     with open(path, 'rb') as f:
#         squad_dict = json.load(f)
# 
#     contexts = []
#     questions = []
#     answers = []
#     for group in squad_dict['data']:
#         for passage in group['paragraphs']:
#             context = passage['context']
#             for qa in passage['qas']:
#                 question = qa['question']
#                 for answer in qa['answers']:
#                     contexts.append(context)
#                     questions.append(question)
#                     answers.append(answer)

#     return contexts, questions, answers

# train_contexts, train_questions, train_answers = read_squad('KorQuAD_v1.0_train.json')
# val_contexts, val_questions, val_answers = read_squad('KorQuAD_v1.0_dev.json')

In [6]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                answer = qa['answers']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('train.json')

In [7]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    guids = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                guid = qa['guid']
                contexts.append(context)
                questions.append(question)
                guids.append(guid)

    return contexts, questions, guids

test_contexts, test_questions, test_guids = read_squad('test.json')

In [8]:
print('훈련 데이터의 본문 개수 :', len(train_contexts))
print('훈련 데이터의 질문 개수 :', len(train_questions))
print('훈련 데이터의 답변 개수 :', len(train_answers))
print('테스트 데이터의 본문 개수 :', len(test_contexts))
print('테스트 데이터의 질문 개수 :', len(test_questions))
print('테스트 데이터의 아이디 개수 :', len(test_guids))

훈련 데이터의 본문 개수 : 17663
훈련 데이터의 질문 개수 : 17663
훈련 데이터의 답변 개수 : 17663
테스트 데이터의 본문 개수 : 4008
테스트 데이터의 질문 개수 : 4008
테스트 데이터의 아이디 개수 : 4008


In [9]:
print('첫번째 샘플의 본문')
print('-----------------')
print(train_contexts[0])

첫번째 샘플의 본문
-----------------
올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.


In [10]:
print('첫번째 샘플의 질문')
print('-----------------')
print(train_questions[0])

첫번째 샘플의 질문
-----------------
북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?


In [11]:
print('첫번째 샘플의 답변')
print('-----------------')
print(train_answers[0])

첫번째 샘플의 답변
-----------------
{'text': '한 달가량', 'answer_start': 478}


In [12]:
print('첫번째 테스트트 데이터터 본문')
print('-----------------')
print(test_contexts[0])

첫번째 테스트트 데이터터 본문
-----------------
BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에디션’을 한정 출시한다고 밝혔다. 이번 BMW 코리아 25주년 에디션(이하 25주년 에디션)은 BMW 3시리즈와 5시리즈, 7시리즈, 8시리즈 총 4종, 6개 모델로 출시되며, BMW 클래식 모델들로 선보인 바 있는 헤리티지 컬러가 차체에 적용돼 레트로한 느낌과 신구의 조화가 어우러진 차별화된 매력을 자랑한다. 먼저 뉴 320i 및 뉴 320d 25주년 에디션은 트림에 따라 옥스포드 그린(50대 한정) 또는 마카오 블루(50대 한정) 컬러가 적용된다. 럭셔리 라인에 적용되는 옥스포드 그린은 지난 1999년 3세대 3시리즈를 통해 처음 선보인 색상으로 짙은 녹색과 풍부한 펄이 오묘한 조화를 이루는 것이 특징이다. M 스포츠 패키지 트림에 적용되는 마카오 블루는 1988년 2세대 3시리즈를 통해 처음 선보인 바 있으며, 보랏빛 감도는 컬러감이 매력이다. 뉴 520d 25주년 에디션(25대 한정)은 프로즌 브릴리언트 화이트 컬러로 출시된다. BMW가 2011년에 처음 선보인 프로즌 브릴리언트 화이트는 한층 더 환하고 깊은 색감을 자랑하며, 특히 표면을 무광으로 마감해 특별함을 더했다. 뉴 530i 25주년 에디션(25대 한정)은 뉴 3시리즈 25주년 에디션에도 적용된 마카오 블루 컬러가 조합된다. 뉴 740Li 25주년 에디션(7대 한정)에는 말라카이트 그린 다크 색상이 적용된다. 잔잔하면서도 오묘한 깊은 녹색을 발산하는 말라카이트 그린 다크는 장식재로 활용되는 광물 말라카이트에서 유래됐다. 뉴 840i xDrive 그란쿠페 25주년 에디션(8대 한정)은 인도양의 맑고 투명한 에메랄드 빛을 연상케 하는 몰디브 블루 컬러로 출시된다. 특히 몰디브 블루는 지난 1993년 1세대 8시리즈에 처음으로 적용되었던 만큼 이를 오마주하는 의미를 담고 있다.


In [13]:
print('첫번째 테스트트 데이터터 질문')
print('-----------------')
print(test_questions[0])

첫번째 테스트트 데이터터 질문
-----------------
말라카이트에서 나온 색깔을 사용한 에디션은?


In [15]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        answer['text'] = answer['text'].rstrip()
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)
        assert context[start_idx:end_idx] == gold_text, "end_index 계산에 에러가 있습니다."
        answer['answer_end'] = end_idx

add_end_idx(train_answers, train_contexts)

In [14]:
print('첫번째 샘플의 답변')
print('-----------------')
print(train_answers[1])

첫번째 샘플의 답변
-----------------
{'text': '한 달', 'answer_start': 478}


In [16]:
train_contexts[0][478]

'한'

In [17]:
train_contexts[0][479]

' '

In [18]:
train_contexts[0][480]

'달'

In [19]:
train_contexts[0][481]

'가'

In [20]:
train_contexts[0][482]

'량'

In [21]:
train_contexts[0][478:483]

'한 달가량'

In [22]:
tokenizer = BertTokenizerFast.from_pretrained('klue/bert-base')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
test_encodings = tokenizer(test_contexts, test_questions, truncation=True, padding=True)

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [24]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    deleting_list = []

    for i in tqdm(range(len(answers))):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
            deleting_list.append(i)

        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
            if i not in deleting_list:
              deleting_list.append(i)

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return deleting_list

In [25]:
deleting_list_for_train = add_token_positions(train_encodings, train_answers)

100%|██████████| 17663/17663 [00:00<00:00, 155301.93it/s]


In [26]:
def delete_samples(encodings, deleting_list):
  input_ids = np.delete(np.array(encodings['input_ids']), deleting_list, axis=0)
  attention_masks = np.delete(np.array(encodings['attention_mask']), deleting_list, axis=0)
  start_positions = np.delete(np.array(encodings['start_positions']), deleting_list, axis=0)
  end_positions = np.delete(np.array(encodings['end_positions']), deleting_list, axis=0)

  X_data = [input_ids, attention_masks]
  y_data = [start_positions, end_positions]

  return X_data, y_data

In [27]:
def for_test(encodings):
  input_ids = np.array(encodings['input_ids'])
  attention_masks = np.array(encodings['attention_mask'])

  X_data = [input_ids, attention_masks]

  return X_data

In [28]:
X_test = for_test(test_encodings)

In [29]:
X_train, y_train = delete_samples(train_encodings, deleting_list_for_train)

In [31]:
# TPU 작동을 위한 코드 TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.126.210.146:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.126.210.146:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [32]:
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [33]:
from transformers import TFBertModel

In [34]:
class TFBertForQuestionAnswering(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertForQuestionAnswering, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.qa_outputs = tf.keras.layers.Dense(2,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                name='qa_outputs')
        self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids, attention_mask=attention_mask)
    
        sequence_output = outputs[0]

        logits = self.qa_outputs(sequence_output)
        start_logits, end_logits = tf.split(logits, 2, axis=-1)

        # start_logits = (batch_size, sequence_length,)
        # end_logits = (batch_size, sequence_length,)
        start_logits = tf.squeeze(start_logits, axis=-1)
        end_logits = tf.squeeze(end_logits, axis=-1)

        start_probs = self.softmax(start_logits)
        end_probs = self.softmax(end_logits)

        return start_probs, end_probs

In [35]:
# 신규
with strategy.scope():
  model = TFBertForQuestionAnswering("klue/bert-base")
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
  model.compile(optimizer=optimizer, loss=loss)

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [36]:
history = model.fit(
    X_train,
    y_train,
    epochs=3,
    verbose=1,
    batch_size=16,
)

Epoch 1/3


1013/1013 [==============================] - 174s 98ms/step - loss: 3.2957 - output_1_loss: 1.5288 - output_2_loss: 1.7669
Epoch 2/3
1013/1013 [==============================] - 99s 97ms/step - loss: 1.7710 - output_1_loss: 0.7855 - output_2_loss: 0.9855
Epoch 3/3
1013/1013 [==============================] - 98s 97ms/step - loss: 1.3513 - output_1_loss: 0.5756 - output_2_loss: 0.7756


In [37]:
# 예측하는 함수
def predict_test_data_by_idx(idx):
  context = tokenizer.decode(X_test[0][idx]).split('[SEP] ')[0]
  question = tokenizer.decode(X_test[0][idx]).split('[SEP] ')[1]
  print('본문 :', context)
  print('질문 :', question)
  output = model([tf.constant(X_test[0][idx])[None, :], tf.constant(X_test[1][idx])[None, :]])
  start = tf.math.argmax(tf.squeeze(output[0]))
  end = tf.math.argmax(tf.squeeze(output[1]))+1
  answer_encoded = X_test[0][idx][start:end]
  print('예측 :',tokenizer.decode(answer_encoded))
  print('----------------------------------------')

In [46]:
tokenizer.decode(X_test[0][3242]).split('[SEP] ')[0]

'[CLS] 미국 의회가 유전자변형 ( GMO ) 식품의 포함 여부를 표시하도록 의무화하는 법안 마련을 추진 중이다. 미국에선 최근 오리건주에서 미승인 GMO 밀이 발견돼 국제적으로 파문이 확산되고 있다. 바버라 박서 민주당 의원 ( 캘리포니아 ) 을 비롯한 미 상원의원 11명은 지난달 말 GMO 식품 및 GMO 원료가 들어간 제품에 GMO 표기를 강제하는 내용의 ‘ 유전자변형 식품 알 권리 법안 ’ 을 발의했다. 또 하원에서도 피터 드파지오 의원 ( 민주당 · 오리건주 ) 주도로 비슷한 내용의 법안이 하원 보건위원회에 제출됐다. 박서 의원은 “ 한국을 비롯해 유럽연합 ( EU ) 일본 등 60개 이상의 핵심 무역 상대국들이 유전자변형 식품의 표기를 법제화했다 ” 며 “ 미국 국민은 건강과 경제, 환경과 종교 등 여러 가지 이유에서 자신들이 소비하는 식품에 유전자변형 기술이 이용됐는지를 알 권리가 있다 ” 고 설명했다. 미국에선 지난달 29일 오리건주의 한 밀밭에서 GMO 밀이 발견돼 논란이 일고 있다. 미국은 세계 최대 밀 수출국이며, 오리건주에서 생산되는 밀의 90 % 가 해외로 수출된다. 일본은 지난달 31일부터 미국산 밀 수입을 잠정 중단했으며, EU도 미국산 밀에 대한 검사를 강화했다. '

In [62]:
# [SEP]가 없는 question에 [SEP]추가
ln = len(test_questions)
for i in tqdm(range(0, ln)):
  question = tokenizer.decode(X_test[0][i]).split('[SEP]')[1]
  if question.endswith('[SEP] '):
    pass
  else:
    question = question + '[SEP] '

100%|██████████| 4008/4008 [00:01<00:00, 2985.09it/s]


In [80]:
import pandas as pd

def save_csv(X_test):
    ln = len(test_questions)
    submission = pd.DataFrame(columns = ['Id','Predicted'])
    for i in tqdm(range(0,ln)):
          output = model([tf.constant(X_test[0][i])[None, :], tf.constant(X_test[1][i])[None, :]])
          start = tf.math.argmax(tf.squeeze(output[0]))
          end = tf.math.argmax(tf.squeeze(output[1]))+1
          answer_encoded = X_test[0][i][start:end]
          submission.loc[i] = [test_guids[i], tokenizer.decode(answer_encoded)]    
    return submission.to_csv('submission.csv', index=False, encoding='utf-8')

save_csv(X_test)

100%|██████████| 4008/4008 [1:06:15<00:00,  1.01it/s]


In [78]:
df1 = pd.DataFrame(columns = ['Id','Predicted'])

In [79]:
new_data = ['0012jfoin03093r', tokenizer.decode(X_test[0][1]).split('[SEP]')[1]]
df1.loc[0] = new_data
df1

,Id,Predicted
0,0012jfoin03093r,사고 비행기의 목적지는?
